In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# FINAL PROJECT: Import DATABASE
# city_data_df = pd.read_csv("IMPORT DATABASE")
# city_data_df.head()
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kieta,PG,-6.2167,155.6333,84.04,72,75,2.98,light rain
1,1,Jamestown,US,42.0970,-79.2353,35.19,90,1,0.00,clear sky
2,2,Avarua,CK,-21.2078,-159.7750,82.45,78,23,14.97,few clouds
3,3,Hirara,JP,24.8000,125.2833,75.47,78,75,12.66,broken clouds
4,4,Cabo San Lucas,MX,22.8909,-109.9124,88.61,39,90,1.99,overcast clouds


In [3]:
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

In [4]:
# ??? ARE WE KEEPING THIS OR PROMPT FOR LOCATION INSTEAD?
# pref_location = float(input("Where would you like to go for your trip? "))
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# FINAL PROJECT: FILTER THE DF USING LOC METHOD
# preferred_location_df = city_data_df.loc[(city_data_df["COUNTRY" OR "CITY"] = pref_location)]
# preferred_cities_df.head(10)
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kieta,PG,-6.2167,155.6333,84.04,72,75,2.98,light rain
2,2,Avarua,CK,-21.2078,-159.7750,82.45,78,23,14.97,few clouds
3,3,Hirara,JP,24.8000,125.2833,75.47,78,75,12.66,broken clouds
4,4,Cabo San Lucas,MX,22.8909,-109.9124,88.61,39,90,1.99,overcast clouds
12,12,Morant Bay,JM,17.8815,-76.4093,80.22,80,99,10.02,overcast clouds
18,18,Tuy Hoa,VN,13.0833,109.3000,76.44,92,100,6.44,overcast clouds
22,22,Marica,BR,-22.9194,-42.8186,75.11,85,27,14.92,scattered clouds
26,26,Vaini,TO,-21.2000,-175.2000,82.56,78,75,11.50,broken clouds
29,29,Port Hedland,AU,-20.3167,118.5667,87.85,58,6,10.36,clear sky
39,39,Zvishavane,ZW,-20.3267,30.0665,76.23,29,4,3.02,clear sky


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kieta,PG,-6.2167,155.6333,84.04,72,75,2.98,light rain
2,2,Avarua,CK,-21.2078,-159.7750,82.45,78,23,14.97,few clouds
3,3,Hirara,JP,24.8000,125.2833,75.47,78,75,12.66,broken clouds
4,4,Cabo San Lucas,MX,22.8909,-109.9124,88.61,39,90,1.99,overcast clouds
12,12,Morant Bay,JM,17.8815,-76.4093,80.22,80,99,10.02,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
675,675,Kupang,ID,-10.1667,123.5833,84.78,79,20,3.44,few clouds
692,692,Kibala,TD,9.1104,18.3463,76.95,88,0,0.00,clear sky
695,695,Tefe,BR,-3.3542,-64.7114,75.22,97,100,1.52,overcast clouds
697,697,Kapoeta,SS,4.7717,33.5903,80.83,48,39,7.00,scattered clouds


In [9]:
# FINAL PROJECT: USE BELOW BUT ADD CURRENT AND PROJECTED COVID CASES
# hotel_df = clean_df[["City", "Country", "Current Covid Cases", "Projected Covid Cases", "Current Description", "Lat", "Lng"]].copy()
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kieta,PG,84.04,light rain,-6.2167,155.6333,
2,Avarua,CK,82.45,few clouds,-21.2078,-159.7750,
3,Hirara,JP,75.47,broken clouds,24.8000,125.2833,
4,Cabo San Lucas,MX,88.61,overcast clouds,22.8909,-109.9124,
12,Morant Bay,JM,80.22,overcast clouds,17.8815,-76.4093,
18,Tuy Hoa,VN,76.44,overcast clouds,13.0833,109.3000,
22,Marica,BR,75.11,scattered clouds,-22.9194,-42.8186,
26,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,
29,Port Hedland,AU,87.85,clear sky,-20.3167,118.5667,
39,Zvishavane,ZW,76.23,clear sky,-20.3267,30.0665,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kieta,PG,84.04,light rain,-6.2167,155.6333,
2,Avarua,CK,82.45,few clouds,-21.2078,-159.7750,
3,Hirara,JP,75.47,broken clouds,24.8000,125.2833,
4,Cabo San Lucas,MX,88.61,overcast clouds,22.8909,-109.9124,
12,Morant Bay,JM,80.22,overcast clouds,17.8815,-76.4093,
...,...,...,...,...,...,...,...
675,Kupang,ID,84.78,few clouds,-10.1667,123.5833,
692,Kibala,TD,76.95,clear sky,9.1104,18.3463,
695,Tefe,BR,75.22,overcast clouds,-3.3542,-64.7114,
697,Kapoeta,SS,80.83,scattered clouds,4.7717,33.5903,


In [12]:
# FINAL PROJECT: CREATE OUTPUT FILE
# output_data_file = "ENTER NEW FILE.csv"
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kieta,PG,84.04,light rain,-6.2167,155.6333,
2,Avarua,CK,82.45,few clouds,-21.2078,-159.7750,
3,Hirara,JP,75.47,broken clouds,24.8000,125.2833,
4,Cabo San Lucas,MX,88.61,overcast clouds,22.8909,-109.9124,
12,Morant Bay,JM,80.22,overcast clouds,17.8815,-76.4093,


In [13]:
# FINAL PROJECT: UPDATE TEMPLATE TO INCLUDE COVID CASES AND PROJECTED COVID CASES
# info_box_template = """
# <dl>
# <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
# <dt>City Name</dt><dd>{City}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# <dt>Current Weather</dt><dd>{Current Description}</dd>
# <dt>Current Covid Cases</dt><dd>{CURRENT COVID CASES}</dd>
# <dt>Projected Covid Cases</dt><dd>{PROJECTED COVID CASES}</dd>
# <dt>Max Temp</dt><dd>{Max Temp} °F</dd>
# </dl>
# """
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))